In [2]:
import json
import pandas
import time
import os
import recordlinkage
import jellyfish as jf
import csv

In [3]:
outputDir = 'scraped_json//'
field_agg = "_"

## Load the source file dict
sourceFiles = json.loads(open('sourceFiles.json', "r").read())

## Load the file schema
sourceSchema = json.loads(open('sourceSchema.json', "r").read())

## Load the id config
idConfig = json.loads(open('idConfig.json', "r").read())

In [4]:
def mergeSourceFiles (source, outputDir, sourceFiles):
    recordList = []
    for file in sourceFiles[source]:  
        file = json.loads(open(outputDir + file, "r", encoding="utf-8").read())
        for record in file:
            recordList.append(record)
    return recordList

In [5]:
## Cycle through all of files per source and drop into a list of dicts
## Printing list names for reference sake
for key in sourceFiles.keys():
    print(key)
    vars()[key] = mergeSourceFiles(key, outputDir, sourceFiles)
    

sports247
rivals
cfbRef
allConf
allAmerican
nflData


In [6]:
## Clean up dirty names
def mungeID(playerString):
    return ''.join(e for e in playerString if e.isalnum()).lower().replace("jr.", "").replace("st.", "state")  

In [7]:
#create a universal identifier
#CHANGE - this is not going to be a list
#CFB-Ref reports it in chronological order so we will join based on position 1

def createNewID (fieldList, thisDict, fieldAgg):
    finalID= ''
    for i in thisDict:
        for idx, val in enumerate(fieldList):
            if (type(i[val]) is list):
                i[val]= mungeID(i[val][0])
                if (len(fieldList) -1 == idx):
                    finalID += str(i[val]).strip('[]').strip("''")
            elif (type(val) is not list):
                i[val] = mungeID(i[val])
                if (len(fieldList) - 1 == idx):
                    finalID += i[val]
                else:
                    finalID = i[val] + fieldAgg
        i['ID'] = finalID
        finalID=''

In [8]:
createNewID(idConfig['sports247'], sports247, field_agg)
createNewID(idConfig['rivals'], rivals, field_agg)
createNewID(idConfig['cfbRef'], cfbRef, field_agg)
createNewID(idConfig['allConf'], allConf, field_agg)
createNewID(idConfig['allAmerican'], allAmerican, field_agg)
createNewID(idConfig['nflData'], nflData, field_agg)

#createNewID(idConfig['rivals'], rivals, field_agg)

In [9]:
df_sports247 = pandas.DataFrame(sports247)
df_rivals = pandas.DataFrame(rivals)
df_cfbRef = pandas.DataFrame(cfbRef)
df_allConf = pandas.DataFrame(allConf)
df_allAmerican = pandas.DataFrame(allAmerican)
df_nflData = pandas.DataFrame(nflData)

#df_sports247.dtypes
df_rivals.dtypes

school           object
year             object
playerName       object
city             object
state            object
highSchool       object
position         object
height          float64
weight          float64
stars             int64
nationalRank    float64
positionRank    float64
stateRank       float64
ID               object
dtype: object

In [19]:
print(len(df_sports247

,school,year,playerName,highSchool,city,state,compRating,position,height,weight,compStars,nationalRank,positionRank,stateRank,ID
0,nebraska,2012,paulthurston,Arvada West,Arvada,CO,0.9407,OT,77,275,4,152,16,2,paulthurston_nebraska
1,nebraska,2012,gregmcmullen,Archbishop Hoban,Akron,OH,0.9099,SDE,77,255,4,269,14,18,gregmcmullen_nebraska
2,nebraska,2012,jordanwesterkamp,Montini Catholic,Lombard,IL,0.9134,WR,73,190,4,254,36,6,jordanwesterkamp_nebraska
3,nebraska,2012,imanicross,North Hall,Gainesville,GA,0.9093,RB,73,222,4,273,15,23,imanicross_nebraska
4,nebraska,2012,tommyarmstrong,Steele,Cibolo,TX,0.9060,DUAL,73,200,4,286,6,42,tommyarmstrong_nebraska
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17059,kentucky,2012,korybrown,East Central C.C.,Decatur,MS,0.8067,S,73,200,3,124,4,20,korybrown_kentucky
17060,kentucky,2012,ryanmosby,Arizona Western College,Yuma,AZ,0.7833,OLB,72,220,2,175,10,19,ryanmosby_kentucky
17061,kentucky,2012,sterlingwright,Coral Reef,Miami,FL,0.7000,S,72,190,2,NA,228,418,sterlingwright_kentucky
17062,kentucky,2012,jeffwitthuhn,Greenwood,Bowling Green,KY,NA,PRO,75,245,0,NA,NA,NA,jeffwitthuhn_kentucky


In [10]:
indexer = recordlinkage.Index()
indexer.block('school')
indexer.block('year')
candidate_links = indexer.index(df_sports247, df_rivals)

In [11]:
print(len(candidate_links))

22388448


In [20]:
c = recordlinkage.Compare()

c.exact('ID', 'ID', label='ID')
c.string('playerName', 'playerName', method='levenshtein', threshold=3, label='playerName')
c.string('city', 'city', method='levenshtein', threshold=3, label='city')
c.string('state', 'state', method='levenshtein', threshold=3, label='state')
c.string('highSchool', 'highSchool', method='levenshtein', threshold=5, label='highSchool')

# The comparison vectors
features = c.compute(candidate_links, df_sports247, df_rivals)

features

ID  playerName  city  state  highSchool
0     19      0         0.0   0.0    0.0         0.0
      34      0         0.0   0.0    0.0         0.0
      43      0         0.0   0.0    0.0         0.0
      51      0         0.0   0.0    0.0         0.0
      55      0         0.0   0.0    0.0         0.0
...          ..         ...   ...    ...         ...
17063 16853   0         0.0   0.0    0.0         0.0
      16855   0         0.0   0.0    0.0         0.0
      16856   0         0.0   0.0    0.0         0.0
      16861   0         0.0   0.0    0.0         0.0
      16877   0         0.0   0.0    0.0         0.0

[22388448 rows x 5 columns]

In [23]:
features.describe()

,ID,playerName,city,state,highSchool
count,2.238845e+07,22388448.0,22388448.0,22388448.0,22388448.0
mean,7.146989e-04,0.0,0.0,0.0,0.0
std,2.672430e-02,0.0,0.0,0.0,0.0
min,0.000000e+00,0.0,0.0,0.0,0.0
25%,0.000000e+00,0.0,0.0,0.0,0.0
50%,0.000000e+00,0.0,0.0,0.0,0.0
75%,0.000000e+00,0.0,0.0,0.0,0.0
max,1.000000e+00,0.0,0.0,0.0,0.0


In [22]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

1.0       16001
0.0    22372447
dtype: int64

In [125]:
ecm = recordlinkage.ECMClassifier()
ecm.fit_predict(feature_vectors)

MultiIndex([], )

In [126]:
print(ecm)

In [127]:
ecm[0]

TypeError: 'ECMClassifier' object is not subscriptable